In [2]:
import pandas as pd
from helper import unzip_gz, zip_gz
import pandas as pd
import numpy as np
import gzip
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
pd.set_option('display.max_columns', None)

In [6]:
with gzip.open('data/qs.crops_20240926.txt.gz', 'rt') as f:
    df = pd.read_csv(f, delimiter='\t')

df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/qs.crops_20240926.txt.gz'

In [32]:
df = df.drop(columns=['COUNTRY_NAME', 'COUNTRY_CODE', 'STATE_NAME', 'STATE_ALPHA',
                      'STATE_FIPS_CODE', 'STATE_ANSI', 'COUNTY_ANSI', 'COUNTY_CODE',
                      'REGION_DESC', 'ZIP_5', 'WATERSHED_DESC', 'CONGR_DISTRICT_CODE',
                      'LOAD_TIME', 'CV_%', 'BEGIN_CODE', 'END_CODE', 'WATERSHED_CODE',
                      'ASD_CODE', 'SOURCE_DESC', 'SECTOR_DESC', 'GROUP_DESC',
                      'UTIL_PRACTICE_DESC', 'WEEK_ENDING', 'COUNTY_NAME', 'ASD_DESC'])
df.head(3)

In [5]:
crops = ["WHEAT", "CORN", "HAY", "SOYBEANS", "BARLEY", "OATS", "SORGHUM", "COTTON",
         "BEANS", "POTATOES", "TOBACCO", "PEANUTS", "SUNFLOWER", "APPLES", "RYE",
         "RICE", "PEACHES", "TOMATOES", "SWEET CORN", "PEAS", "PEARS", "SUGARBEETS",
         "MELONS", "PECANS", "BLUEBERRIES", "SQUASH", "ONIONS", "LETTUCE", "PEPPERS",
         "GRAPES", "SWEET POTATOES", "CHERRIES", "STRAWBERRIES", "CABBAGE", "CUCUMBERS",
         "ORANGES", "RASPBERRIES", "PUMPKINS", "BLACKBERRIES", "PLUMS & PRUNES", "CARROTS",
         "BROCCOLI", "SUGARCANE", "GARLIC", "ASPARAGUS", "SPINACH", "BEETS", "GRAPEFRUIT",
         "EGGPLANT", "APRICOTS", "CAULIFLOWER", "LEMONS"]

In [6]:
filtered_df = df[df['COMMODITY_DESC'].isin(crops)]

### Filter the df to only the 52 crops in the crops list

In [7]:
not_units = ['OPERATIONS', 'PCT FAIR', 'PCT VERY POOR', '$ / BU', '$',
             '$ / TON', '$ / CWT', 'ACRES', 'PCT POOR', '$ / BOX, PHD EQUIV',
             'PCT', '$ / BOX, ON TREE EQUIV', 'NUMBER', 'SQ FT', 'CENTS / BU',
             'PCT OF PARITY', 'INDEX', '$, PHD EQUIV', '$ / BOX, FOB',
             'PCT BY SIZE GROUP', 'PCT BY GRADE', 'PCT NONE', 'PCT LIGHT',
             'PCT MODERATE', 'POINTS / LB', 'PCT OF FARM SALES',
             'PCT OF TOTAL STOCKS', 'CUERDAS', 'PCT ACTIVE', 'PCT OF FARM OPERATIONS',
             '$ / BOX', '$ / TON, DRY BASIS' ,'REACTIONS SINCE 1984', 'PCT OF PRODUCTION',
             'PCT OF MKTG YEAR', '$ / LB', ]

for item, value in filtered_df['UNIT_DESC'].value_counts().items():
    if value <= 500:
        not_units.append(item)

In [31]:
df = filtered_df.loc[~filtered_df['UNIT_DESC'].isin(not_units)]

for key, value in df['UNIT_DESC'].value_counts().items():
    print(key,  value)

### Check if all 50 states are in the LOCATION_DESC column

In [30]:
all_states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida",
    "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine",
    "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska",
    "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota",
    "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee",
    "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"
]

zip_df = pd.read_csv("data/uszips.csv")
zip_df

In [10]:
zip_df = zip_df[zip_df['state_name'].isin(all_states)]
zip_df = zip_df[['zip', 'state_name']]

In [11]:
zip_df['zip'] = zip_df['zip'].astype(str)

zip_to_state = dict(zip(zip_df['zip'], zip_df['state_name']))

capitalized_states = [item.upper() for item in all_states]

In [29]:
def replace_region_with_state(location):
    """Function that checks if state name is in LOCATION_DESC if the name contains a state name but is not just
    the state name it converts it into only the state name. For example OREGON SOUTHWEST DOUGLAS becomes OREGON"""
    for state in capitalized_states:
        if state in location:
            return state

    if location in zip_to_state:
        print(f"Zip:{location} State:{zip_to_state[location].upper()}")
        return zip_to_state[location].upper()

    print(f"NONE {location}")
    return "NONE"

df['LOCATION_DESC'] = df['LOCATION_DESC'].apply(replace_region_with_state)
df['LOCATION_DESC'].value_counts()

In [13]:
location_list = []

for key in df['LOCATION_DESC'].value_counts().keys():
    location_list.append(key)
len(location_list)

50

In [28]:
for location in df['LOCATION_DESC'].value_counts().keys():
    print(location)

In [27]:
df = df[df['LOCATION_DESC'] != "NONE"]
df['LOCATION_DESC'].value_counts()

In [16]:
row, column = df.shape
print(f"Row: {row:,}  Column:{column}")

Row: 7,131,741  Column:14


In [17]:
df = df[(df['YEAR'] >= 1980) & (df['YEAR'] <= 2023)]

In [25]:
no_info_crops = []
for crop, amount in df['COMMODITY_DESC'].value_counts().items():
    if amount <= 1500:
        no_info_crops.append(crop)

df = df.loc[~df['COMMODITY_DESC'].isin(no_info_crops)]
df['COMMODITY_DESC'].value_counts()

In [26]:
df = df.drop(columns=['CLASS_DESC', 'CLASS_DESC', 'STATISTICCAT_DESC', 'SHORT_DESC',
                      'DOMAIN_DESC', 'DOMAINCAT_DESC', 'AGG_LEVEL_DESC', 'FREQ_DESC',
                      'REFERENCE_PERIOD_DESC', 'PRODN_PRACTICE_DESC'])
df.head()

In [20]:
df['VALUE'] = pd.to_numeric(df['VALUE'], errors='coerce')
df = df.dropna(subset=['VALUE'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2731136 entries, 16 to 22309639
Data columns (total 5 columns):
 #   Column          Dtype  
---  ------          -----  
 0   COMMODITY_DESC  object 
 1   UNIT_DESC       object 
 2   LOCATION_DESC   object 
 3   YEAR            int64  
 4   VALUE           float64
dtypes: float64(1), int64(1), object(3)
memory usage: 125.0+ MB


In [21]:
# Apply log transformation to 'VALUE'
# Use np.log1p to handle zero values safely
df['log_value'] = np.log1p(df['VALUE'])

# Compute group-wise lower and upper percentiles on log-transformed data
# Here, we're using the 5th and 95th percentiles
df['lower_percentile'] = df.groupby(['COMMODITY_DESC', 'UNIT_DESC'])['log_value'].transform(lambda x: x.quantile(0.05))
df['upper_percentile'] = df.groupby(['COMMODITY_DESC', 'UNIT_DESC'])['log_value'].transform(lambda x: x.quantile(0.95))

# Clip log-transformed values to the percentile range
df['clipped_log_value'] = df['log_value'].clip(lower=df['lower_percentile'], upper=df['upper_percentile'])

# Apply the normalization formula
df['normalized_value'] = (
    (df['clipped_log_value'] - df['lower_percentile']) /
    (df['upper_percentile'] - df['lower_percentile'])
) * 100

# Handle division by zero when lower_percentile == upper_percentile
equal_percentile_mask = df['upper_percentile'] == df['lower_percentile']
df.loc[equal_percentile_mask, 'normalized_value'] = df.loc[equal_percentile_mask, 'VALUE'].apply(lambda x: 100 if x != 0 else 0)

# Clip normalized values to be between 0 and 100
df['normalized_value'] = df['normalized_value'].clip(lower=0, upper=100)

/Users/masongalusha/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [22]:
print(f"{df.shape[0]:,}")

2,731,136


In [23]:
df = df.drop(columns=['log_value', 'lower_percentile', 'upper_percentile', 'clipped_log_value', 'VALUE'])
df.head()

,COMMODITY_DESC,UNIT_DESC,LOCATION_DESC,YEAR,normalized_value
16,SOYBEANS,BU / ACRE,INDIANA,2014,92.499630
35,HAY,TONS / ACRE,KANSAS,2001,50.044943
36,SUGARBEETS,PCT GOOD,IDAHO,2022,55.783833
45,COTTON,PCT EXCELLENT,CALIFORNIA,2010,37.311430
48,HAY,TONS / ACRE,MINNESOTA,1989,33.524074


In [24]:
zip_gz(df, "data/clean_crop.gz")